### Connectome Analysis

Notebook accompanying Zarkali et al. Dementia risk in Parkinson’s disease is associated with interhemispheric connectivity loss and determined by regional gene expression. NeuroImage Clinical 2020

Author: A Zarkali
Last updated: 30 June 2020

In [2]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import shapiro
from scipy.stats import percentileofscore
import statsmodels.api as sm
#import statsmodels.formula.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
from pathlib import Path
import brainconn as con
import bct as bct
import networkx as nx

# Enable inline plotting
%matplotlib inline

Failed to import duecredit due to No module named 'duecredit'


In [1]:
### Read in the clinical data
df = pd.read_excel(r"Data/Participants.csv")
participantList = df.Participants

### Define modules

1) First calculate a group average connectome for all participants (can be found in the repository data folder as "Average_Connectome.csv")

2) Run the Community Louvain algorithm (1000 iterations, gamma=1 and replicate using gamma=0.8 and gamma=1.3)

In [ ]:
### 1) Calculate the group average connectome 

array = np.zeros((379, 379))
data_folder = Path(r"Data\Connectomes")

for i in range(0, len(participantList)):
    path = data_folder / participantList[i] / "connectome_norm.csv"
    data = pd.read_csv(path, header=None, sep=" ")
    array = array + data
array = array / len(participantList)
np.savetxt("Average_Connectome.csv", array, delimiter=",")

In [ ]:
### 2) Run Community Louvain algorithm

from functions import module_detection ## load the relevant function from the accompanying file
gamma = 1.0 ## for replication analyses change to 0.8 and 1.3 respectively

x,y = module_detection(r"Data\Connectomes\Average_Connectome.csv", gamma, 1000)
np.savetxt("ModuleAllocation_gamma1.txt", x, delimiter=",")
np.savetxt("Louvain_vectors_gamma1.txt", y, delimiter= ",")

### Calculate connectome density per participant

This is already calculated and provided as a data column ("Density") in Data/Participants.csv

In [ ]:
#### Calculate connectome density for each participant
### this has been merged on Data/Participants.csv as the Density data column

## Find connectome density per connectome
data_folder = Path(r"Data\Connectomes")
list = []
for i in range(len(participantList)):
    path = data_folder / participantList[i] / "connectome_norm.csv"
    data = pd.read_csv(path, header=None, sep=" ")
    den, x, y = con.physical_connectivity.density_und(data)
    list.append(den)
np.savetxt("Density.csv", list, delimiter=",")

In [ ]:
### Compare density amongst groups

data = pd.read_csv(r"Data\Participants.csv")
mean = data.Density.mean()
r, p = stats.pearsonr(data.PD, data.Density)

#### return mean density, pearson correlation coefficient and p value
mean, r, p 

### Calculate strength and path length per connection type

He we will calculate the strength and path length of different connection types:
- Subcortical-cortical
- Intrahemispheric
- Interhemispheric
- Intramodular

In [ ]:
### Input module allocations
##### For the glasser atlas our module allocations for gamma = 1.0 are found in Data/Modules/ as text files
##### You can adjust this process for different number of modules and different atlases
modulePath = r"Data/Modules/"

##### Load all modules
bgLeft = np.loadtxt((modulePath + "BGLeft.txt"), dtype="int16")
bgLeft[:] = [x - 1 for x in bgLeft] ## as indexing of connectomes starts at 0
bgRight = np.loadtxt((modulePath + "BGRight.txt"), dtype="int16")
bgRight[:] = [x - 1 for x in bgRight]
module1 = np.loadtxt((modulePath + "Module1.txt"), dtype="int16")
module1[:] = [x - 1 for x in module1]
module1 = np.loadtxt((modulePath + "Module1.txt"), dtype="int16")
module1[:] = [x - 1 for x in module1]
module2 = np.loadtxt((modulePath + "Module2.txt"), dtype="int16")
module2[:] = [x - 1 for x in module2]
module3 = np.loadtxt((modulePath + "Module3.txt"), dtype="int16")
module3[:] = [x - 1 for x in module3]
module4 = np.loadtxt((modulePath + "Module4.txt"), dtype="int16")
module4[:] = [x - 1 for x in module4]
module5 = np.loadtxt((modulePath + "Module5.txt"), dtype="int16")
module5[:] = [x - 1 for x in module5]
module6 = np.loadtxt((modulePath + "Module6.txt"), dtype="int16")
module6[:] = [x - 1 for x in module6]
module7 = np.loadtxt((modulePath + "Module7.txt"), dtype="int16")
module7[:] = [x - 1 for x in module7]
module8 = np.loadtxt((modulePath + "Module8.txt"), dtype="int16")
module8[:] = [x - 1 for x in module8]

#### Input hemispheres (modules 1-4 are Left, modules 5-8 are right)
cortexLeft = module1 + module2 + module3 + module4
cortexRight = module5 + module6 + module7 + module8

In [5]:
### Calculate strength 
from functions import module_connections ## import relevant function from the accompanying functions.py 

sumStrength = [ ] ## empty list to hold data
data_folder = Path(r"Data/Connectomes") ## root data folder

# Loop through all subjects
# Need to run this once for each connection type 
#### Example here for interhemispheric connections between module 1 and module 5
for i in range(len(participantList)):
    path = data_folder / participantList[i] / "connectome_norm.csv" 
    graph = pd.read_csv(path, header=None, sep=" ")
    x = module_connections(graph, module1, module5)
    sumStrength.append(x)
np.savetxt("Strength\StrengthInter1_5.txt", sumStrength, delimiter=" ")

In [ ]:
### Calculate path length
from functions import module_length ## import relevant function from the accompanying functions.py 

sumStrength = [ ] ## empty list to hold data
data_folder = Path(r"Data/Connectomes") ## root data folder

# Loop through all subjects
# Need to run this once for each connection type 
#### Example here for interhemispheric connections between module 1 and module 5
for i in range(len(participantList)):
    path = data_folder / participantList[i] / "new_connectome.csv" ### need the length connectomes
    graph = pd.read_csv(path, header=None, sep=" ")
    x = module_length(graph, module1 module5)
    sumStrength.append(x)
np.savetxt("Length\LengthInter1_5.txt", sumStrength, delimiter=" ")

In [ ]:
### Merge files to a single df
#### Example to merge all txt files from the calculated path length and strenght to a single csv file
import os
import glob
os.chdir(r"Strength/") ### This is the directory that holds all the txt files you want to merge
extension = 'txt'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f, header=None) for f in all_filenames], axis=0)
#export to csv
combined_csv.to_csv("Strength_Combined.csv", index=False)

The Strength and Length combined csv files can be used for between-group comparisons in combination with the Participants.csv.

To do this we used statsmodels' Mixed Linear model (MixedLM.from_formula) with age and gender as covariates

### Calculate atrophy rates

Here we will transform the connection strength values for each participant to atrophy rates, compared to controls. 

First we will calculate a z score for each connection.

Then we will tahn transform.

In [ ]:
# Calculate atrophy rates compared to controls
## Load the average control connectome
average_path = Path(r"Data\Connectomes\Average_Connectome.csv") ## the mean strength per connection for the group-averaged control connectome
std_path = Path(r"Data\Connectomes\STD_Connectome.csv") ## the standard deviation per connection for the group-averaged control connectome
average_connectome = np.genfromtxt(average_path, delimiter=",")
std_connectome = np.genfromtxt(std_path, delimiter=",")

data_folder = Path(r"Data\Connectomes")
for i in range(len(participantList)):
    path = data_folder / participantList[i] / "connectome_norm.csv"
    ## Load connectomes
    data = np.genfromtxt(path)
    # Calculate z score
    data_z = (average_connectome - data) / std_connectome
    export_name = str(participantList[i]) + "_Zscore.txt"
    np.savetxt(export_name, data_z, delimiter=" ")

In [ ]:
# Perform tahn transform for each participant
data_folder = Path(r"Data\Connectomes")
average_z = np.zeros((379,379)) # create an empty array to hold results
pdList = df[df.PD == 1].Participants # load a list of PD participants only
## Loop through all PD participants and perform tahn transform
for i in range(len(pdList)):
    path = data_folder / str(pdList[i] + "_Zscore.txt")
    data = np.genfromtxt(path)
    transform = np.tanh(data)
    export_name = str(pdList[i] + "_tahn.txt")
    np.savetxt(export_name, transform, delimiter=" ")
## These can then be grouped to calculate average tahn transformed values across different groups (ie PD vs controls etc)